In [5]:
with open('tasks/19_test.txt', 'r') as f:
    data = f.read().splitlines()

In [6]:
data

['Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.',
 'Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian.']

In [ ]:
class Factory:
    def __init__(self, ore, caly, obsidian, geode):
        self.ore = 1
        self.clay = 0
        self.obsidian = 0
        self.goede = 0